### 2.1 필요한 라이브러리 설치

In [41]:
!pip install geopandas folium pysal spopt

### 2.2 데이터 준비
- 자전거 대여소의 위치 데이터와 유동인구, 자전거 이용량 등의 데이터를 준비합니다.


In [48]:
from spopt.locate import PMedian
import geopandas as gpd
from shapely.geometry import Point

# 자전거 대여소 데이터 (위도, 경도, 유동인구)
data = {
    '대여소명': ['망원역 1번출구 앞', '합정역 2번출구 앞', '홍대입구역 3번출구 앞'],
    '위도': [37.556, 37.550, 37.558],
    '경도': [126.910, 126.914, 126.923],
    '유동인구': [1000, 1500, 1200]
}

# DataFrame 생성
df = pd.DataFrame(data)

# GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(df['경도'], df['위도'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

### 2.3 최적화 모델 정의 (Location-Allocation Model)
- spopt 라이브러리를 사용하여 시설 입지 문제를 해결하는 모델을 정의할 수 있습니다. 여기서는 Location-Allocation 문제를 풀어 자전거 대여소의 최적 위치를 선정합니다.

In [54]:
# 수요 지점과 시설 후보지로 나누기 (여기서는 같은 데이터프레임을 사용하지만 실제로는 다를 수 있음)
gdf_demand = gdf[['geometry', '유동인구']]  # 수요 지점 (유동인구 포함)
gdf_fac = gdf[['geometry']]                 # 시설 후보지

# P-median 모델 생성 (대여소 개수 설정)
p_median = PMedian.from_geodataframe(
    gdf_demand,
    gdf_fac,
    demand_col="geometry",       # 수요 지점의 좌표
    facility_col="geometry",     # 대여소 후보지의 좌표
    weights_cols="유동인구",     # 각 수요 지점의 가중치 (유동인구)
    p_facilities=2               # 선택할 대여소 개수
)

# 모델 실행
p_median.solve()

# 결과 출력 (선정된 대여소)
selected_facilities = gdf_fac.iloc[p_median.facility_array_]
print(selected_facilities)

TypeError: PMedian.solve() missing 1 required positional argument: 'solver'

### 2.4 결과 시각화- 
Folium을 사용하여 선정된 대여소 위치를 지도에 시각화합니다.

In [ ]:
import folium

# 지도 생성 (서울 중심 좌표 설정)
m = folium.Map(location=[37.550, 126.920], zoom_start=13)

# 기존 대여소 표시
for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['대여소명']}\n유동인구: {row['유동인구']}\n이용량: {row['이용량']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# 선택된 최적 대여소 표시 (초록색 마커)
for idx, row in selected_facilities.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"선정된 대여소",
        icon=folium.Icon(color='green')
    ).add_to(m)

# 지도 출력
m.save('optimal_bikeshare_map.html')
m